In [1]:
import cifar_gpu_fun
reload(cifar_gpu_fun)
from cifar_gpu_fun import *
import bcd
reload(bcd)
from robust_probability import ctype_robust_probability
%pylab inline

Using gpu device 0: Tesla K20c (CNMeM is enabled with initial size: 98.0% of memory, CuDNN 5005)
/data/vaishaal/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your CuDNN version is more recent then Theano. If you see problems, try updating Theano or downgrading CuDNN to version 4.
  warnings.warn(warn)
/data/vaishaal/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Populating the interactive namespace from numpy and matplotlib


#### Define Some Constants 

In [2]:
NUM_FEATURE_BATCHES=2
RHO = [0.001, 0.01, 0.1]
DATA_BATCH_SIZE=(1280)
FEATURE_BATCH_SIZE=(1024)
NUM_TRAIN = 50000
NUM_TEST = 10000
NUM_CLASSES = 10
NUM_FEATURES = FEATURE_BATCH_SIZE * NUM_FEATURE_BATCHES * 8
POOL_TYPE ='avg'
FILTER_GEN ='empirical'
BANDWIDTH = 1.0
LAMBDAS = [0.01, 0.1, 0.34, 1.0, 3.37, 10.0, 100.0, 337.0, 1000.0][::-1]
CUDA_CONVNET = True
SCALE = 55.0
BIAS = 1.25
MIN_VAR_TOL = 1e-4
TOT_FEAT = FEATURE_BATCH_SIZE*NUM_FEATURE_BATCHES

In [3]:
(XTrain, labelsTrain), (XTest, labelsTest) = load_cifar_processed()
patches = patchify_all_imgs(XTrain, (6,6), pad=False)
if FILTER_GEN == 'gaussian':
       filter_gen = make_gaussian_filter_gen(1.0)
elif FILTER_GEN == 'empirical':
        filter_gen = make_empirical_filter_gen(patches, labelsTrain)
else:
    raise Exception('Unknown FILTER_GEN value')

(50000, 3, 32, 32)


In [4]:
# Featurize here
X = np.vstack((XTrain, XTest))
(XFeaturized, filters) = conv(X, filter_gen, 1024, NUM_FEATURE_BATCHES, 1280, CUDA_CONVNET, pool_size=14, symmetric_relu=True)
XFeaturizedTrain = XFeaturized[:50000,:,:,:]                                                                                                                                             
XFeaturizedTest = XFeaturized[50000:,:,:,:]

XFeaturizedTrain = XFeaturizedTrain.reshape(50000, -1)
XFeaturizedTest = XFeaturizedTest.reshape(10000, -1)

Out Shape  2 x 2 x 4096
FEATURE BATCH # 0 DATA BATCH # 0  SIZE IS  1280
FEATURE BATCH # 0 DATA BATCH # 1  SIZE IS  1280
FEATURE BATCH # 0 DATA BATCH # 2  SIZE IS  1280
FEATURE BATCH # 0 DATA BATCH # 3  SIZE IS  1280
FEATURE BATCH # 0 DATA BATCH # 4  SIZE IS  1280
FEATURE BATCH # 0 DATA BATCH # 5  SIZE IS  1280
FEATURE BATCH # 0 DATA BATCH # 6  SIZE IS  1280
FEATURE BATCH # 0 DATA BATCH # 7  SIZE IS  1280
FEATURE BATCH # 0 DATA BATCH # 8  SIZE IS  1280
FEATURE BATCH # 0 DATA BATCH # 9  SIZE IS  1280
FEATURE BATCH # 0 DATA BATCH # 10  SIZE IS  1280
FEATURE BATCH # 0 DATA BATCH # 11  SIZE IS  1280
FEATURE BATCH # 0 DATA BATCH # 12  SIZE IS  1280
FEATURE BATCH # 0 DATA BATCH # 13  SIZE IS  1280
FEATURE BATCH # 0 DATA BATCH # 14  SIZE IS  1280
FEATURE BATCH # 0 DATA BATCH # 15  SIZE IS  1280
FEATURE BATCH # 0 DATA BATCH # 16  SIZE IS  1280
FEATURE BATCH # 0 DATA BATCH # 17  SIZE IS  1280
FEATURE BATCH # 0 DATA BATCH # 18  SIZE IS  1280
FEATURE BATCH # 0 DATA BATCH # 19  SIZE IS  1280
FEATUR

In [ ]:
XFeaturizedTrain.dot(XFeaturizedTrain.T)

In [5]:
# Set weights here
W = np.ones(XTrain.shape[0])[:, np.newaxis]

#W = np.random.dirichlet(2*np.ones(XTrain.shape[0]))[:, np.newaxis] * XTrain.shape[0]

In [6]:
# Solve weighted least squares here (in primal)
%time model = learnPrimal(XFeaturizedTrain, labelsTrain, W)

cifar_gpu_fun.py:201: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if (W == None):


X SHAPE  (50000, 16384)
Computing XTX
Done Computing XTX
CPU times: user 6min 16s, sys: 2min 11s, total: 8min 27s
Wall time: 33.8 s


In [7]:
# Evaluate on training set
YHat = XFeaturizedTrain.dot(model)
labelsPredTrain = np.argmax(YHat, axis=1)

In [8]:
# Calculate training loss and training accuracy
YTrain = np.eye(max(labelsTrain) + 1)[labelsTrain]
loss = np.linalg.norm(YHat - YTrain)**2 + LAMBDAS[0]*(np.linalg.norm(model)**2)
train_acc = metrics.accuracy_score(labelsTrain, labelsPredTrain)
print "Training Loss: ", loss
print "Training Accuracy ", train_acc

Training Loss:  721935.18898
Training Accuracy  0.97116


In [9]:
# Evaluate on validation set
YHatTest = XFeaturizedTest.dot(model)
labelsPredTest = np.argmax(YHatTest, axis=1)
test_acc = metrics.accuracy_score(labelsTest, labelsPredTest)
print "Testing Accuracy: ", test_acc

Testing Accuracy:  0.7323


In [26]:
W.sum()

49999.99999999992

# Robust solve

In [ ]:
# Start weights
N = XTrain.shape[0]
W = np.ones(N)[:, np.newaxis]


for lambda_ in LAMBDAS:
    for rho_ in RHO:
        print lambda_
        print rho_
        for i in xrange(5):
            print "Iteration: ", i
            # This should converge quickly right?

            model = learnPrimal(XFeaturizedTrain, labelsTrain, W, reg=lambda_)
            # Evaluate on training set
            YHat = XFeaturizedTrain.dot(model)
            labelsPredTrain = np.argmax(YHat, axis=1)

            # Calculate training loss and training accuracy
            YTrain = np.eye(max(labelsTrain) + 1)[labelsTrain]
            losses = np.square(YHat - YTrain).sum(axis=1)
            W = N*ctype_robust_probability.robust_probability(-losses, rho_)[:, np.newaxis]
            print W
            loss = np.linalg.norm(YHat - YTrain)**2 + LAMBDAS[0]*(np.linalg.norm(model)**2)
            train_acc = metrics.accuracy_score(labelsTrain, labelsPredTrain)
            print "Training Loss: ", loss
            print "Training Accuracy ", train_acc

            # Evaluate on validation set
            YHatTest = XFeaturizedTest.dot(model)
            labelsPredTest = np.argmax(YHatTest, axis=1)
            test_acc = metrics.accuracy_score(labelsTest, labelsPredTest)
            print "Testing Accuracy: ", test_acc

1000.0
0.001
Iteration:  0
X SHAPE  (50000, 16384)
Computing XTX
Done Computing XTX
[[ 0.96756863]
 [ 1.03388408]
 [ 0.96529532]
 ..., 
 [ 1.01355031]
 [ 1.05421067]
 [ 0.94572711]]
Training Loss:  29152.9456496
Training Accuracy  0.75166
Testing Accuracy:  0.7012
Iteration:  1
X SHAPE  (50000, 16384)
Computing XTX
Done Computing XTX
[[ 0.96719032]
 [ 1.03418515]
 [ 0.96485671]
 ..., 
 [ 1.01436895]
 [ 1.05409542]
 [ 0.94549088]]
Training Loss:  29163.1204308
Training Accuracy  0.74838
Testing Accuracy:  0.6981
Iteration:  2
X SHAPE  (50000, 16384)
Computing XTX
